## Read tabular data from a CSV

We're going to use the [City of Philadelphia's employee salary dataset](https://www.opendataphilly.org/dataset/employee-salaries-overtime) to explore `pandas`


In this case the CSV path happens to be [a URL coming from Philly's open data portal](https://phl.carto.com/api/v2/sql?filename=employee_earnings&format=csv&skipfields=cartodb_id,the_geom,the_geom_webmercator&q=SELECT%20*%20FROM%20employee_earnings), although a regular filepath would also work: `C:\Users\yourname\Desktop\some_file.csv`

In [1]:
import pandas as pd

data_url = "https://phl.carto.com/api/v2/sql?filename=employee_earnings&format=csv&skipfields=cartodb_id,the_geom,the_geom_webmercator&q=SELECT%20*%20FROM%20employee_earnings"

In [6]:
raw_df = pd.read_csv(data_url)

## Explore the dataframe

`df.dtypes` will show you the column names and the type of data contained within each column

In [7]:
raw_df.dtypes

calendar_year                      int64
quarter                            int64
last_name                         object
first_name                        object
title                             object
job_code                          object
department_name                   object
department_number                  int64
base_salary                      float64
salary_type                       object
overtime_gross_pay_qtd           float64
base_gross_pay_qtd               float64
longevity_gross_pay_qtd          float64
post_separation_gross_pay_qtd    float64
miscellaneous_gross_pay_qtd      float64
employee_category                 object
compulsory_union_code             object
termination_month                float64
termination_year                 float64
dtype: object

You can also operate on a single column if needed using square brackets. The code below selects all values in the `department_name` column and reduces it to only the unique entries.

In [25]:
raw_df["department_name"].unique()

array(['OIT Ofc of Innovation and Tech',
       'COM Commerce - Division of Aviation',
       'OPA Ofc of Property Assessment', 'DHS Dept of Human Services',
       'PWD Water', 'FJD 1st Judicial District PA', 'PPS Prisons',
       'PPD Police', 'MAP Mural Arts Program',
       'OSH Ofc of Homeless Services', 'PPR Parks and Recreation',
       'DPH Health', 'LNI Licenses and Inspections', 'REC Records',
       'PFD Fire', 'MDO Managing Director Office', 'STS Streets',
       'PCC City Council', 'DAO District Attorney',
       'CEO Community Empowerment Ofc', 'OHR Office of Human Resources',
       'OFM Fleet Management', 'LAW Law Dept', 'REV Revenue', 'MAY Mayor',
       'FLP Free Library of Phila', 'BPR Board of Pensions Retiremt',
       'DPP Public Property', 'CMS City Commissioners', 'SHF Sheriff',
       'CLR City Controller', 'DBH Behav Hlth ID Svcs',
       'ROW Register of Wills', 'REP City Representative',
       'CAO Ofc of Chief Admin Officer', 'FIN Finance',
       'OLR Off

## Analyze the data

Let's filter the dataset to all employees in city council, and then summarize by calculating the average salary for each job title.

In [22]:
df = raw_df.copy()

df = df[df["department_name"] == "PCC City Council"]

df = df[["title", "base_salary"]]

df.head()

,title,base_salary
52,Chief Of Staff,97850.0
59,Legislative Assistant,40000.0
91,Chief Of Staff,108150.0
179,Legislative Director,100476.0
343,Intern,NaN


In [50]:
summary = df.groupby(["title"]).agg(['count','mean']).sort_values([("base_salary", "mean")], ascending=False)

for idx, row in summary.iterrows():
    salary_formatted_like_money = "${:,.2f}". format(row[("base_salary", "mean")])
    sample_size = row[("base_salary", "count")]

    print(idx, "\n\t-->", salary_formatted_like_money, "\t n=", sample_size)

President Of City Council 
	--> $168,622.29 	 n= 7.0
Senior Legislative Counsel 
	--> $155,285.71 	 n= 7.0
Senior Legislative Advisor 
	--> $149,210.00 	 n= 7.0
Majority Leader 
	--> $143,946.00 	 n= 7.0
Majority Whip 
	--> $141,204.14 	 n= 7.0
Exec Dir Housing & Community Dev 
	--> $140,996.00 	 n= 7.0
Minority Leader 
	--> $139,945.00 	 n= 3.0
Chief Financial Officer 
	--> $138,287.50 	 n= 8.0
Majority Deputy Whip 
	--> $136,938.75 	 n= 8.0
Minority Whip 
	--> $135,869.00 	 n= 3.0
Councilmember 
	--> $134,404.38 	 n= 87.0
Chief Operating Officer 
	--> $132,594.50 	 n= 8.0
Director Of Administrative Services 
	--> $128,685.00 	 n= 7.0
Senior Policy Advisor 
	--> $119,395.45 	 n= 11.0
Chief Clerk 
	--> $114,376.43 	 n= 7.0
Chief Ethics Officer 
	--> $111,450.00 	 n= 7.0
Chief Of Staff 
	--> $103,149.12 	 n= 84.0
Legistar System Admin 
	--> $98,112.00 	 n= 4.0
Legislative Director 
	--> $97,721.20 	 n= 10.0
Director Of Special Projects 
	--> $92,700.00 	 n= 7.0
Digital Media Director 
	

## Explore additional aspects of the data

Looks like there's 7 entries for `President Of City Council`, when you'd expect there to only be one.

Let's filter the raw dataframe to this job title and inspect the rows. Looks like there's one entry per quarter, starting Q2 2019 through Q4 2020.

In [39]:
raw_df[raw_df["title"] == "President Of City Council"]

,calendar_year,quarter,last_name,first_name,title,job_code,department_name,department_number,base_salary,salary_type,overtime_gross_pay_qtd,base_gross_pay_qtd,longevity_gross_pay_qtd,post_separation_gross_pay_qtd,miscellaneous_gross_pay_qtd,employee_category,compulsory_union_code,termination_month,termination_year
25728,2019,2,Clarke,Darrell,President Of City Council,P400,PCC City Council,1,167118.0,Salaried,NaN,37846.62,NaN,NaN,0.0,Exempt,L,NaN,NaN
39004,2019,3,Clarke,Darrell,President Of City Council,P400,PCC City Council,1,167118.0,Salaried,NaN,44698.92,NaN,NaN,0.0,Exempt,L,NaN,NaN
93421,2019,4,Clarke,Darrell,President Of City Council,P400,PCC City Council,1,167118.0,Salaried,NaN,38271.30,NaN,NaN,0.0,Exempt,L,NaN,NaN
101776,2020,1,Clarke,Darrell,President Of City Council,P400,PCC City Council,1,167118.0,Salaried,NaN,44649.85,NaN,NaN,0.0,Exempt,L,NaN,NaN
126516,2020,2,Clarke,Darrell,President Of City Council,P400,PCC City Council,1,170628.0,Salaried,NaN,38271.30,NaN,NaN,0.0,Exempt,L,NaN,NaN
174904,2020,3,Clarke,Darrell,President Of City Council,P400,PCC City Council,1,170628.0,Salaried,NaN,45603.43,NaN,NaN,0.0,Exempt,L,NaN,NaN
197800,2020,4,Clarke,Darrell,President Of City Council,P400,PCC City Council,1,170628.0,Salaried,NaN,45762.36,NaN,NaN,0.0,Exempt,L,NaN,NaN
